In [1]:
#Langchain for using frontier models 
%pip install -qU langchain langchain-openai langchain-anthropic langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [2]:
#importing tester from src directory 
import sys 
sys.path.append('..')
from src.advanced_tester import Tester

In [3]:
#loading the test and train data 
import pickle

train_path = '/Users/vishesh/projects/Pricer/data/balanced/train.pkl'
test_path = '/Users/vishesh/projects/Pricer/data/balanced/test.pkl'
with open(train_path,'rb') as f:
    train = pickle.load(f)
with open(test_path,'rb') as f:
    test = pickle.load(f)

In [4]:
from langchain.chat_models import init_chat_model

In [5]:
import re
def extract_price(text):
    text = text.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", text)
    return float(match.group()) if match else 0

In [ ]:

models = {
    # OpenAI Models
    'gpt_4o': init_chat_model("gpt-4o", model_provider="openai"),
    'gpt_5_mini': init_chat_model("gpt-5-mini", model_provider="openai"),
    'gpt_5': init_chat_model("gpt-5", model_provider="openai"),
    'gpt_5_nano': init_chat_model("gpt-5-nano", model_provider="openai"),
    'gpt_4_1': init_chat_model("gpt-4.1", model_provider="openai"),
    'gpt_4o_mini': init_chat_model("gpt-4o-mini", model_provider="openai"),
    
    # Anthropic Models
    'claude_opus_4': init_chat_model("claude-opus-4-20250514", model_provider="anthropic"),
    'claude_sonnet_4_5': init_chat_model("claude-sonnet-4-5", model_provider="anthropic"),
    'claude_opus_4_1': init_chat_model("claude-opus-4-1", model_provider="anthropic"),
    'claude_sonnet_4_0': init_chat_model("claude-sonnet-4-0", model_provider="anthropic"),
    'claude_3_7_sonnet_latest': init_chat_model("claude-3-7-sonnet-latest", model_provider="anthropic"),
    
    # Google Gen-AI Models
    'gemini_2_5_flash': init_chat_model("gemini-2.5-flash", model_provider="google_genai"),
    'gemini_2_5_pro': init_chat_model("gemini-2.5-pro", model_provider="google_genai"),
    'gemini_2_5_flash_lite': init_chat_model("gemini-2.5-flash-lite", model_provider="google_genai"),
    'gemini_2_0_flash': init_chat_model("gemini-2.0-flash", model_provider="google_genai")
}


In [7]:

prefix ='''Complete this sentence with only the price of the product , 
            do not explain give simple plain text of what you think the price of the product is'''


In [8]:
def create_pricer(model):
    def pricer(datapoint):
        content = prefix + datapoint['test_prompt']
        response = model.invoke(content).content
        price = extract_price(response)
        return price if price is not None else 0.0
    return pricer


In [9]:
# dynamic pricer function
pricers = {name: create_pricer(model) for name, model in models.items()}

In [ ]:
# Running tests
for name, pricer in pricers.items():
    print(f"Testing {name}...")
    Tester.test(pricer, test)